## **Parameter Optimization of Boosting Models**
> Gradient Boosting Machine (GBM), XGBoost (XGB), LightGBM(LGBM), CatBoost (GB)의 적절한 Parameter setting 값 찾기

In [1]:
!pip install catboost

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.6/98.6 MB 5.5 MB/s eta 0:00:00


In [9]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier

from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV

In [3]:
!git clone https://github.com/yun-seon/DX-Training-LGES-0614.git

Cloning into 'DX-Training-LGES-0614'...
remote: Enumerating objects: 75, done.
remote: Counting objects: 100% (14/14), done.
remote: Compressing objects: 100% (14/14), done.
remote: Total 75 (delta 4), reused 0 (delta 0), pack-reused 61
Unpacking objects: 100% (75/75), 15.63 MiB | 6.06 MiB/s, done.


In [6]:
# import dataset

data = pd.read_csv('/content/DX-Training-LGES-0614/Data_wafer_sensor_classification.csv')

del data['Unnamed: 0']
data.head()

,0,1,2,3,4,5,6,7,8,9,...,711,712,713,714,715,716,717,718,719,y
0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,...,-97.0,9.436896e-16,0.0,-1.000000e+00,6.505213e-18,1.517883e-18,-2.710505e-20,0.000000e+00,0.000000e+00,0
1,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,...,-96.0,-9.600000e+01,-6.0,6.938894e-17,5.551115e-17,-1.000000e+00,-1.734723e-18,-2.059984e-18,1.084202e-19,0
2,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,...,-97.0,-3.774758e-15,-1.0,-1.000000e+00,-1.000000e+00,-1.000000e+00,-1.000000e+00,5.551115e-17,-1.000000e+00,0
3,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,...,-116.0,-1.160000e+02,-116.0,-1.160000e+02,-1.160000e+02,-2.000000e+00,-1.000000e+00,1.387779e-17,-3.469447e-18,0
4,2.0,3.0,3.0,3.0,2.0,2.0,2.0,2.0,2.0,2.0,...,-115.0,-1.150000e+02,-115.0,-1.150000e+02,-1.150000e+02,-1.150000e+02,-1.150000e+02,-1.150000e+02,-1.150000e+02,0


In [8]:
# split train and test data (test ratio: 30%)
X_train, X_test, Y_train, Y_test = train_test_split(data.iloc[:, 0:720], data.iloc[:,720], test_size = 0.3)

In [10]:
# 성능 비교를 위한 Boosting 모델 정의
lreg = LogisticRegression()
gbm = GradientBoostingClassifier()
xgb = XGBClassifier()
lgb = LGBMClassifier()
cb = CatBoostClassifier()

In [11]:
# 최적의 파라미터 값 찾기 위한 범위 설정

# Logistic
param_lreg = { 'C' : [1.0, 3, 5, 7, 10],
              'max_iter': [50, 200, 100, 300, 500,700, 800]
              }

# GBM
param_gbm = {"max_depth" : [4,5,6,7,8,9,10],
             "learning_rate" : [0.01,0.1,0.2,0.3,0.4,0.5],
             "n_estimators" : [100,200,300,500]
              }

# XGB
param_xgb = {"max_depth" : [4,5,6,7,8,9,10],
             "learning_rate" : [0.01,0.1,0.2,0.3,0.4,0.5],
             "n_estimators" : [100,200,300,500]
}

# LGB
param_lgb = {"learning_rate" : [0.01,0.1,0.2,0.3,0.4,0.5],
             "max_depth" : [25, 50, 75],
             "num_leaves" : [100,300,500,900,1200],
             "n_estimators" : [100, 200, 300,500,800,1000],
             "learning_rate" : [0.01,0.1,0.2,0.3,0.4,0.5]
              }

# CB
param_cb = {'iterations': [100, 150, 200],
           'learning_rate': [0.03, 0.1],
           'depth': [2, 4, 6, 8],
           'l2_leaf_reg': [0.2, 0.5, 1, 3]
}

In [ ]:
# 최적의 파라미터 찾기 위한 GridsearchCV 실행
# cv = StratifiedKFold(n_splits=5, shuffle = True, random_state=42)
gscv_lreg = GridSearchCV (estimator = lreg, param_grid = param_lreg, scoring ='accuracy', cv = 3, refit=True, n_jobs=1, verbose=2)
gscv_gbm = GridSearchCV (estimator = gbm, param_grid = param_gbm, scoring ='accuracy', cv = 3, refit=True, n_jobs=1, verbose=2)
gscv_xgb = GridSearchCV (estimator = xgb, param_grid = param_xgb, scoring ='accuracy', cv = 3, refit=True, n_jobs=1, verbose=2)
gscv_lgb = GridSearchCV (estimator = lgb, param_grid = param_lgb, scoring ='accuracy', cv = 3, refit=True, n_jobs=1, verbose=2)
gscv_cb = GridSearchCV (estimator = cb, param_grid = param_cb, scoring ='accuracy', cv = 3, refit=True, n_jobs=1, verbose=2)

# GridSearch 모델 학습
gscv_lreg.fit(X_train, Y_train)
gscv_gbm.fit(X_train, Y_train)
gscv_xgb.fit(X_train, Y_train)
gscv_lgb.fit(X_train, Y_train)
gscv_cb.fit(X_train, Y_train)

Fitting 3 folds for each of 35 candidates, totalling 105 fits
[CV] END .................................C=1.0, max_iter=50; total time=   0.1s


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _c

[CV] END .................................C=1.0, max_iter=50; total time=   0.1s
[CV] END .................................C=1.0, max_iter=50; total time=   0.1s


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _c

[CV] END ................................C=1.0, max_iter=200; total time=   0.2s


...

[CV] END .................................C=10, max_iter=700; total time=   0.6s
[CV] END .................................C=10, max_iter=800; total time=   0.5s


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


[CV] END .................................C=10, max_iter=800; total time=   0.7s
[CV] END .................................C=10, max_iter=800; total time=   0.8s


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Fitting 3 folds for each of 168 candidates, totalling 504 fits
[CV] END ..learning_rate=0.01, max_depth=4, n_estimators=100; total time=   6.2s
[CV] END ..learning_rate=0.01, max_depth=4, n_estimators=100; total time=   6.2s
[CV] END ..learning_rate=0.01, max_depth=4, n_estimators=100; total time=   7.3s
[CV] END ..learning_rate=0.01, max_depth=4, n_estimators=200; total time=  10.0s
[CV] END ..learning_rate=0.01, max_depth=4, n_estimators=200; total time=  12.5s
[CV] END ..learning_rate=0.01, max_depth=4, n_estimators=200; total time=  12.3s
[CV] END ..learning_rate=0.01, max_depth=4, n_estimators=300; total time=  15.5s
[CV] END ..learning_rate=0.01, max_depth=4, n_estimators=300; total time=  18.5s
[CV] END ..learning_rate=0.01, max_depth=4, n_estimators=300; total time=  20.4s
[CV] END ..learning_rate=0.01, max_depth=4, n_estimators=500; total time=  29.3s
[CV] END ..learning_rate=0.01, max_depth=4, n_estimators=500; total time=  34.7s
[CV] END ..learning_rate=0.01, max_depth=4, n_

In [ ]:
print("="*30)
print('Lreg 파라미터: ', gscv_lreg.best_params_)
print('Lreg 예측 정확도: {:.4f}'.format(gscv_lreg.best_score_))

print("="*30)
print('GBM 파라미터: ', gscv_gbm.best_params_)
print('GBM 예측 정확도: {:.4f}'.format(gscv_gbm.best_score_))

print("="*30)
print('XGB 파라미터: ', gscv_xgb.best_params_)
print('XGB 예측 정확도: {:.4f}'.format(gscv_xgb.best_score_))

print("="*30)
print('LGB 파라미터: ', gscv_lgb.best_params_)
print('LGB 예측 정확도: {:.4f}'.format(gscv_lgb.best_score_))

print("="*30)
print('CB 파라미터: ', gscv_cb.best_params_)
print('CB 예측 정확도: {:.4f}'.format(gscv_cb.best_score_))
print("="*30)